# **Phase 3**

In [54]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow.keras import optimizers

# Set random seed for reproducability
tensorflow.random.set_seed(0)

print("Libraries imported.")
print('Keras version:',keras.__version__)
print('TensorFlow version:',tensorflow.__version__)

Libraries imported.
Keras version: 3.7.0
TensorFlow version: 2.18.0


In [56]:
TRAIN_PATH = 'train_data.csv'
TEST_PATH = 'test_data.csv'

In [57]:
train = pd.read_csv('train_data.csv')

In [58]:
test = pd.read_csv('test_data.csv')

In [59]:
X_train = train.drop(columns=['isFraud'])
y_train = train['isFraud']
X_train.shape, y_train.shape

((10167052, 15), (10167052,))

In [60]:

X_test = test.drop(columns=['isFraud'])
y_test = test['isFraud']
X_test.shape, y_test.shape

((1272524, 15), (1272524,))

In [61]:
X_train, X_unused, y_train, y_unused = train_test_split(X_train, y_train, test_size=0.95, stratify=y_train, random_state=777)

In [63]:
X_test, X_unused, y_test, y_unused = train_test_split(X_test, y_test, test_size=0.90, stratify=y_test, random_state=777)

In [64]:
# Set data types for float features
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Set data types for categorical labels
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)
print('Ready...')

Ready...


In [65]:
X_train.shape

(508352, 15)

In [66]:
X_test.shape

(127252, 15)

### Define Neural Network

In [67]:
num_features = X_train.shape[1]
num_classes = 2

In [68]:
# Define a classifier network
hl_neurons = 20 # Number of hidden layer nodes

mlp = Sequential()
mlp.add(Dense(hl_neurons, input_dim=num_features, activation='relu'))
mlp.add(Dense(hl_neurons, input_dim=hl_neurons, activation='relu'))
mlp.add(Dense(num_classes, input_dim=hl_neurons, activation='softmax'))

print(mlp.summary())

c:\Users\n01715077\Documents\prepstone-final\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 20)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │            42 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 782 (3.05 KB)

 Trainable params: 782 (3.05 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
#hyper-parameters
lr = 0.001
num_epochs = 50

opt = optimizers.Adam(learning_rate=lr)

mlp.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['recall'])

# Train the model over 50 epochs using 10-observation batches and using the test holdout dataset for validation

history = mlp.fit(X_train, y_train, epochs=num_epochs, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/50
7943/7943 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.6928 - recall: 0.5020 - val_loss: 0.6843 - val_recall: 0.9987
Epoch 2/50
7943/7943 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.6928 - recall: 0.5007 - val_loss: 0.6843 - val_recall: 0.9987
Epoch 3/50
7943/7943 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.6928 - recall: 0.5007 - val_loss: 0.6843 - val_recall: 0.9987
Epoch 4/50
7943/7943 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.6928 - recall: 0.5007 - val_loss: 0.6843 - val_recall: 0.9987
Epoch 5/50
7943/7943 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.6928 - recall: 0.5007 - val_loss: 0.6843 - val_recall: 0.9987
Epoch 6/50
7943/7943 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 0.6928 - recall: 0.5007 - val_loss: 0.6843 - val_recall: 0.9987
Epoch 7/50
7943/7943 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.6928 - recall: 0.5007 - val_loss: 0.6843 - val_recall: 0.9987
Epoch 8/50
7943/7943 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.6928 - recall: 0.5007 - val_loss: 0.6843 - val